In [9]:
movies = pd.read_csv('./ml-latest-small/movies.csv') 
print(movies.head(5))

movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [7]:
import pandas as pd
ratings = pd.read_csv('./ml-latest-small/ratings.csv')    
print(ratings.head(20)) 

userId  movieId  rating  timestamp
0        1        1     4.0  964982703
1        1        3     4.0  964981247
2        1        6     4.0  964982224
3        1       47     5.0  964983815
4        1       50     5.0  964982931
5        1       70     3.0  964982400
6        1      101     5.0  964980868
7        1      110     4.0  964982176
8        1      151     5.0  964984041
9        1      157     5.0  964984100
10       1      163     5.0  964983650
11       1      216     5.0  964981208
12       1      223     3.0  964980985
13       1      231     5.0  964981179
14       1      235     4.0  964980908
15       1      260     5.0  964981680
16       1      296     3.0  964982967
17       1      316     3.0  964982310
18       1      333     5.0  964981179
19       1      349     4.0  964982563


In [18]:
data = pd.merge(movies, ratings, on='movieId')
print(data.head(5))
data = data[['userId','rating','movieId','title']].sort_values('userId').to_csv('./ml-latest-small/data.csv', index=False, encoding='utf8')
files = pd.read_csv('./ml-latest-small/data.csv')
print(files.head(5))

movieId             title                                       genres  \
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
1        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
2        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
3        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
4        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   

   userId  rating   timestamp  
0       1     4.0   964982703  
1       5     4.0   847434962  
2       7     4.5  1106635946  
3      15     2.5  1510577970  
4      17     4.5  1305696483  
   userId  rating  movieId                       title
0       1     4.0        1            Toy Story (1995)
1       1     4.0     1777  Wedding Singer, The (1998)
2       1     5.0     1732    Big Lebowski, The (1998)
3       1     3.0     1676    Starship Troopers (1997)
4       1     5.0       50  Usual Suspects, The (1995)


In [23]:
import csv

# read data.csv
content = []
with open('./ml-latest-small/data.csv', encoding='utf8') as fp:  
    cr = csv.reader(fp)
    for row in cr:
        # print(row)
        content.append(row)

# from list to dictionary
data = {}
for line in content[1:100]:
    line = line.strip().split(',')
    print(line)
    if not line[0] in data.keys():
        data[line[0]] = {line[3]:line[1]}
    else:
        data[line[0]][line[3]] = line[1]
print(data)

keys (a.k.a. 12 Monkeys) (1995)']
['610', '3.5', '40815', 'Harry Potter and the Goblet of Fire (2005)']
['610', '3.0', '2628', 'Star Wars: Episode I - The Phantom Menace (1999)']
['610', '2.0', '5476', 'Halloween: Resurrection (Halloween 8) (2002)']
['610', '4.0', '2901', 'Phantasm (1979)']
['610', '4.0', '83134', 'Tucker & Dale vs Evil (2010)']
['610', '4.0', '79501', 'Deadly Outlaw: Rekka (a.k.a. Violent Fire) (Jitsuroku Andô Noboru kyôdô-den: Rekka) (2002)']
['610', '3.5', '40723', 'Wolf Creek (2005)']
['610', '4.0', '3727', 'Near Dark (1987)']
['610', '3.0', '118702', 'Unbroken (2014)']
['610', '3.5', '39801', 'Great Yokai War, The (Yôkai daisensô) (2005)']
['610', '3.0', '82641', 'One-Armed Swordsman, The (Dubei dao) (1967)']
['610', '0.5', '61818', 'Crow, The: Wicked Prayer (2005)']
['610', '5.0', '5954', '25th Hour (2002)']
['610', '3.0', '62849', 'RocknRolla (2008)']
['610', '4.0', '5953', 'Devils on the Doorstep (Guizi lai le) (2000)']
['610', '3.5', '5956', 'Gangs of New York